In [17]:
import tensorflow as tf
from tensorflow import keras

In [18]:
tf.config.run_functions_eagerly(True)

In [19]:
cifar10 = keras.datasets.cifar10

In [20]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [21]:
x_train.shape

(50000, 32, 32, 3)

In [22]:
x_train = x_train/255
x_test = x_test/255

In [23]:
from keras.utils import np_utils

In [24]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [25]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,BatchNormalization,MaxPooling2D,Conv2D,Dropout

In [26]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(32,32,3)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  model.summary()
  
  return model

In [27]:
!pip install keras-tuner

In [29]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [34]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='cifar10_output_2',project_name="Cifar10")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                802848    
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 813,322
Trainable params: 813,322
Non-trainable params: 0
__________________________________________________

In [35]:
tuner_search.search(x_train,y_train,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 01m 02s]
val_accuracy: 0.10239999741315842

Best val_accuracy So Far: 0.5893999934196472
Total elapsed time: 00h 05m 05s
INFO:tensorflow:Oracle triggered exit


In [36]:
model=tuner_search.get_best_models(num_models=1)[0]

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        4864      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        76848     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3096688   
_________________________________________________________________
flatten_1 (Flatten)          (None, 112)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,179,530
Trainable params: 3,179,530
Non-trainable params: 0
______________________________________________

In [39]:
model.fit(x_train, y_train, epochs=50, validation_split=0.1, initial_epoch=3)

Epoch 4/50
   9/1407 [..............................] - ETA: 18s - loss: 0.1226 - accuracy: 0.9583

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


1407/1407 [==============================] - 20s 14ms/step - loss: 0.1856 - accuracy: 0.9356 - val_loss: 2.5395 - val_accuracy: 0.5584
Epoch 5/50
1407/1407 [==============================] - 20s 14ms/step - loss: 0.1463 - accuracy: 0.9505 - val_loss: 2.9223 - val_accuracy: 0.5588
Epoch 6/50
1407/1407 [==============================] - 21s 15ms/step - loss: 0.1358 - accuracy: 0.9534 - val_loss: 2.9626 - val_accuracy: 0.5598
Epoch 7/50
1407/1407 [==============================] - 20s 14ms/step - loss: 0.1212 - accuracy: 0.9602 - val_loss: 3.3598 - val_accuracy: 0.5662
Epoch 8/50
1407/1407 [==============================] - 20s 14ms/step - loss: 0.1062 - accuracy: 0.9646 - val_loss: 3.4584 - val_accuracy: 0.5680
Epoch 9/50
1407/1407 [==============================] - 21s 15ms/step - loss: 0.0967 - accuracy: 0.9685 - val_loss: 3.5864 - val_accuracy: 0.5604
Epoch 10/50
1407/1407 [==============================] - 21s 15ms/step - loss: 0.0886 - accuracy: 0.9712 - val_loss: 3.7744 - val_accur

In [40]:
  prediction_score = model.evaluate(x_test, y_test, verbose=0)

  print('Test Loss and Test Accuracy', prediction_score)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Test Loss and Test Accuracy [7.881112575531006, 0.5418999791145325]
